In [2]:
import ccxt
import time
import concurrent.futures

# Replace with your actual API keys for Bybit and Bitget
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to Bitget API
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.1 / 100  # 0.1% for testing
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds

# Adjustable trading fees per exchange
TRADING_FEES = {
    'Bybit': 0.001,   # 0.1%
    'Bitget': 0.0008  # 0.08%
}

exchanges = {'Bybit': bybit, 'Bitget': bitget}

# Cache futures markets
futures_markets_cache = {}

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol]

        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    try:
        ticker = exchange.fetch_ticker(symbol)
        return ticker.get("last", None)
    except Exception:
        return None

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") and ":USDT" in f]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, exchange_name, symbol):
    futures_list = futures_markets_cache.get(exchange_name, [])
    futures_symbol = find_futures_symbol(symbol, futures_list)

    if not futures_symbol:
        return

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_spot = executor.submit(get_price, exchange, symbol)
        future_futures = executor.submit(get_price, exchange, futures_symbol)
        spot_price = future_spot.result()
        futures_price = future_futures.result()

    if not spot_price or not futures_price:
        return

    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)

    if net_profit >= SPREAD_THRESHOLD:
        print(f"🔍 Arbitrage Opportunity on {exchange_name}: {symbol}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print(f"    ⚡ Execute arbitrage on {exchange_name}")
        print("-" * 60)

def process_tokens(exchange, exchange_name, tokens):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_intra_exchange_arbitrage, exchange, exchange_name, token) for token in tokens]
        concurrent.futures.wait(futures)

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()


✅ Found 437 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
🔍 Arbitrage Opportunity on Bybit: XEM/USDT
    ✅ Spot Price: 0.01934 USDT
    ✅ Futures Price: 0.01942 USDT
    ✅ Spread: 0.4137%
    ✅ Net Profit After Fees: 0.2137%
    ⚡ Execute arbitrage on Bybit
------------------------------------------------------------
🔍 Arbitrage Opportunity on Bybit: OM/USDT
    ✅ Spot Price: 6.66070 USDT
    ✅ Futures Price: 6.69900 USDT
    ✅ Spread: 0.5750%
    ✅ Net Profit After Fees: 0.3750%
    ⚡ Execute arbitrage on Bybit
------------------------------------------------------------
✅ Found 552 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...
🔍 Arbitrage Opportunity on Bitget: LUCE/USDT
    ✅ Spot Price: 0.00965 USDT
    ✅ Futures Price: 0.00969 USDT
    ✅ Spread: 0.4145%
    ✅ Net Profit After Fees: 0.2545%
    ⚡ Execute arbitrage on Bitget
------------------------------------------------------------
🔍 Arbitrage Opportunity on Bitget: HI